In [2]:
import pandas as pd
import torch
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import TensorDataset, DataLoader
import os
import cv2 as cv
from torch.utils.data import Dataset
from torchvision import transforms
import torch.nn.functional as F
from PIL import Image
import torch.nn as nn
from torch.optim import Adam
import matplotlib.pyplot as plt
import numpy as np
from torch.autograd import Variable

# Define the data and label

In [3]:
def getlabel(dir):
    labels=[]
    for name in os.listdir(dir):
        if os.path.splitext(name)[1] == '.png':
            fname=os.path.splitext(name)[0]
            labels.append(fname)
    labels = list(map(int, labels))
    labels = torch.tensor(labels)
    # Normalize labels
    #min = torch.min(labels)
    #max = torch.max(labels)
    #labels = (labels - min) / (max - min)
    labels = labels.type(torch.FloatTensor)
    return labels

def generate_dataset(dir):
    """
    set_label should be 'torch.tensor([1])' if two-catogory and positive sample
    """
    train_data = []
    for file_name in os.listdir(dir):
        if file_name != "Thumbs.db":
            img_dir = os.path.join(dir, file_name)
            img = cv.imread(img_dir)
            img = cv.resize(img, (769, 432))   # /5 resize img
            #img_gray = cv.cvtColor(img,cv.COLOR_RGB2GRAY)
            pimg = Image.fromarray(img)
            train_data.append(pimg)
    return train_data

In [4]:
traindir= './train/'
validdir = './test/'
train_data0 = generate_dataset(traindir)
train_label0=getlabel(traindir)
valid_data0 = generate_dataset(validdir)
valid_label0 = getlabel(validdir)


In [34]:
train_label0

tensor([  0.,   1.,  10., 100., 101., 102., 103., 104., 105., 106., 107., 108.,
        109.,  11., 110., 111., 112., 113., 114., 115., 116., 117., 118., 119.,
         12., 120., 122., 123., 124., 125., 126., 129.,  13., 130., 132., 133.,
        134., 135., 136., 137., 138., 139.,  14., 140., 141., 142., 143., 144.,
        145., 146., 147., 148., 149.,  15., 150., 151., 152., 153., 154., 155.,
        156., 157., 159.,  16., 160., 162., 163., 164., 165., 166., 167., 168.,
        169.,  17., 170., 171., 172., 173., 174., 175., 176., 177., 178., 179.,
         18., 180., 182., 183., 184., 185., 186., 189.,  19., 190., 191., 192.,
        193., 194., 195., 196., 197., 198., 199.,   2.,  20., 200., 201., 203.,
        204., 205.,  21.,  22.,  23.,  24.,  25.,  26.,  27.,  28.,  29.,   3.,
         30.,  32.,  33.,  34.,  35.,  36.,  37.,  38.,  39.,   4.,  43.,  44.,
         45.,  46.,  47.,  48.,  49.,   5.,  50.,  51.,  52.,  53.,  54.,  55.,
         56.,  57.,  58.,   6.,  60.,  6

# 重写dataset类

In [5]:
class MyDataset(Dataset):
    def __init__(self, data, labels, transform=None, target_transform=None):
        self.transform = transform
        self.target_transform = target_transform
        imgs = []
        for i in range(len(labels)):
            # print(type(data[i]))    # <class 'PIL.Image.Image'>
            im_tensor = transform(data[i])#.to(torch.device("cpu"))
            imgs.append((im_tensor, labels[i]))
        self.imgs = imgs                         # DataLoader通过getitem读取图片数据
    def __getitem__(self, index):
        fn, label = self.imgs[index]
        return fn, label
    def __len__(self):
        return len(self.imgs)
    

# 用MyDataset构建数据集

In [6]:
normMean = [0.5, 0.5, 0.5]
normStd = [0.5, 0.5, 0.5]
transform = transforms.Compose(
    [transforms.Grayscale(num_output_channels=1), #彩色图像转灰度图像num_output_channels默认1
    transforms.ToTensor(),  # range [0, 255] -> [0.0,1.0]   
    #transforms.Normalize(normMean, normStd)
    ])

# 也可以再定义train_transform加入一些数据增强 
train_data = MyDataset(train_data0, train_label0, transform=transform)
valid_data = MyDataset(valid_data0, valid_label0, transform=transform)
train_loader = DataLoader(dataset=train_data, batch_size=1, shuffle=True)
valid_loader = DataLoader(dataset=valid_data, batch_size=1, shuffle=True)
dataiter=iter(train_loader)
images, labels = dataiter.next()
print(images.shape)
print(labels.shape)

torch.Size([1, 1, 432, 769])
torch.Size([1])


In [7]:
len(train_label0)

187

# 定义网络

In [7]:
class Net(torch.nn.Module):
    # Net类的初始化函数
    def __init__(self, n_feature, n_hidden, n_output):
        # 继承父类的初始化函数
        super(Net, self).__init__()
        # 网络的隐藏层创建，名称可以随便起
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=3, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(3)
        self.conv2 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(6)
        self.pool = nn.MaxPool2d(2,2)
        self.fc1 = nn.Linear(6*108*192, n_feature)
        self.fc2 = nn.Linear(n_feature, n_hidden)
        # 输出层(预测层)创建，接收来自隐含层的数据
        self.predict_layer = nn.Linear(n_hidden, n_output)

    # 网络的前向传播函数，构造计算图
    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))      #432-3+1=430,430/2=215;769-3+1=767,767/2=384
        x = self.pool(x)
        x = F.relu(self.bn2(self.conv2(x)))                #215-3+1=213,213/2=108;384-3+1=382,382/2=192
        x = self.pool(x)   
        x = x.view(-1, 6*108*192)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        # 用relu函数处理隐含层输出的结果并传给输出层
        x = F.relu(x)
        predict_result = self.predict_layer(x)
        return predict_result

In [8]:
def testAccuracy():
    
    net.eval()
    accuracy = 0.0
    total = 0.0
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    #device = torch.device("cpu")

    with torch.no_grad():
        for data in valid_loader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            # run the model on the test set to predict labels
            outputs = net(images)
            # the label with the highest energy will be our prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            accuracy += (predicted == labels).sum().item()
    
    # compute the accuracy over all test images
    accuracy = (100 * accuracy / total)
    return(accuracy)

def train(num_epochs):
    
    best_accuracy = 0.0

    # Define your execution device
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    #device = torch.device("cpu")

    # Convert model parameters and buffers to CPU or Cuda
    net.to(device)
    for epoch in range(num_epochs):  # loop over the dataset multiple times
        running_loss = 0.0
        running_acc = 0.0

        for i, (images, labels) in enumerate(train_loader, 0):
            #images, labels = dataiter.next()
            # 输入数据进行预测
            # get the inputs
            images = Variable(images.to(device))
            labels = Variable(labels.to(device))
            prediction = net(images)
            print(prediction,labels)
            print(prediction.data.cpu().numpy()[0])
            print(labels.data.cpu().numpy())
            # 计算预测值与真值误差，注意参数顺序问题
            # 第一个参数为预测值，第二个为真值
            loss = loss_func(prediction, labels)

            # 开始优化步骤
            # 每次开始优化前将梯度置为0
            optimizer.zero_grad()
            # 误差反向传播
            loss.backward()
            # 按照最小loss优化参数
            optimizer.step()
            print('epoch:',i,'loss:',loss.item())
            if i % 1000 == 999:    
                # print every 1000 (twice per epoch) 
                print('[%d, %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / 1000))
                # zero the loss
                running_loss = 0.0
            # if i % 2 == 0:
        #     # plot and show learning process
        #         plt.cla()
        #         # 无误差真值曲线
        #         plt.plot(labels.cpu().numpy(), labels.cpu().numpy(), c='blue', lw='3')
        #         # 实时预测的曲线
        #         plt.scatter(labels.data.cpu().numpy()[0], prediction.data.cpu().numpy()[0])
        #         plt.text(0.5, 0, 'Loss=%.4f' % loss.data.cpu().numpy(), fontdict={'size': 20, 'color':  'red'})
        #         plt.pause(0.1)
        # # 计算准确率
        # # Compute and print the average accuracy fo this epoch when tested over all 10000 test images
        accuracy = testAccuracy()
        print('For epoch', epoch+1,'the test accuracy over the whole test set is %d %%' % (accuracy))


    print('Finished Training')
        #可视化训练结果
        # if i % 2 == 0:
        #     # 清空上一次显示结果
        #     plt.cla()
        #     # 无误差真值曲线
        #     plt.plot(images.numpy(), labels.numpy(), c='blue', lw='3')
        #     # 有误差散点
        #     plt.scatter(images.numpy(), labels.numpy(), c='orange')
        #     # 实时预测的曲线
        #     plt.plot(images.numpy(), prediction.data.numpy(), c='red', lw='2')
        #     plt.text(-0.5, -65, 'Time=%d Loss=%.4f' % (i, loss.data.numpy()), fontdict={'size': 15, 'color': 'red'})
        #     plt.pause(0.1)

In [9]:
Vdataiter=iter(valid_loader)
vimg, vlabels = Vdataiter.next()
print(vimg.shape)
print(vlabels.shape)
print(vlabels)

torch.Size([1, 1, 432, 769])
torch.Size([1])
tensor([31.])


## 预测结果

In [10]:
def predict(model, device):
    Vdataiter=iter(valid_loader)
    vimg, vlabels = Vdataiter.next()
    model.to(device)
    with torch.no_grad():
        vimg=vimg.to(device)
        out = model(vimg)
        #_, pre = torch.max(out.data, 1)
        return out, vlabels

# 定义模型

In [17]:
# 输入输出的数据维度，这里都是1维
INPUT_FEATURE_DIM = 1000
# 隐含层中神经元的个数
NEURON_NUM = 100
OUTPUT_FEATURE_DIM = 1
# 学习率，越大学的越快，但也容易造成不稳定，准确率上下波动的情况
LEARNING_RATE = 0.001

# 定义模型
net = Net(n_feature=INPUT_FEATURE_DIM, n_hidden=NEURON_NUM, n_output=OUTPUT_FEATURE_DIM)
print(net)
# 训练网络
# 这里也可以使用其它的优化方法
optimizer = torch.optim.Adam(net.parameters(), lr=LEARNING_RATE)
# 定义一个误差计算方法
loss_func = torch.nn.MSELoss() # 定义交叉熵损失函数 交叉熵损失函数是用来衡量两个概率分布之间的距离的#nn.MSELoss()
# Define the loss function with Classification Cross-Entropy loss and an optimizer with Adam optimizer

Net(
  (conv1): Conv2d(1, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(3, 6, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=124416, out_features=1000, bias=True)
  (fc2): Linear(in_features=1000, out_features=100, bias=True)
  (predict_layer): Linear(in_features=100, out_features=1, bias=True)
)


# 主函数

In [18]:
# Vdataiter=iter(valid_loader)
# vimg, vlabels = Vdataiter.next()
# print(vimg.shape)
# print(vlabels.shape)
# print(vlabels)
if __name__ == "__main__":

    torch.cuda.empty_cache()
    
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    #device = torch.device("cpu")
    # Let's build our model
    train(5)
    print('Finished Training')

    # Test which classes performed well
    testAccuracy()
    
    # Let's load the model we just created and test the accuracy per label
    # model = net()
    # path = "myFirstModel.pth"
    # model.load_state_dict(torch.load(path))

    print(predict(net, device))


tensor([[0.0714]], device='cuda:0', grad_fn=<AddmmBackward0>) tensor([150.], device='cuda:0')
[0.07137006]
[150.]


c:\Users\brcy\anaconda3\envs\pytorch-cy1\lib\site-packages\torch\nn\modules\loss.py:530: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


epoch: 0 loss: 22478.595703125
tensor([[68.0156]], device='cuda:0', grad_fn=<AddmmBackward0>) tensor([175.], device='cuda:0')
[68.015564]
[175.]
epoch: 1 loss: 11445.669921875
tensor([[177.8111]], device='cuda:0', grad_fn=<AddmmBackward0>) tensor([164.], device='cuda:0')
[177.81114]
[164.]
epoch: 2 loss: 190.7476348876953
tensor([[260.7398]], device='cuda:0', grad_fn=<AddmmBackward0>) tensor([35.], device='cuda:0')
[260.73978]
[35.]
epoch: 3 loss: 50958.4453125
tensor([[210.2189]], device='cuda:0', grad_fn=<AddmmBackward0>) tensor([90.], device='cuda:0')
[210.21892]
[90.]
epoch: 4 loss: 14452.587890625
tensor([[142.8755]], device='cuda:0', grad_fn=<AddmmBackward0>) tensor([87.], device='cuda:0')
[142.87555]
[87.]
epoch: 5 loss: 3122.076904296875
tensor([[82.6978]], device='cuda:0', grad_fn=<AddmmBackward0>) tensor([93.], device='cuda:0')
[82.69781]
[93.]
epoch: 6 loss: 106.13517761230469
tensor([[43.1611]], device='cuda:0', grad_fn=<AddmmBackward0>) tensor([43.], device='cuda:0')
[43.1

In [15]:
print(predict(net, device))

NameError: name 'predict' is not defined

# save the Net

In [12]:
torch.save(net, 'net.pkl')

# Load the Net

In [13]:
def restore_net():
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    net = torch.load('net.pkl').cuda()
    print(predict(net, device))


In [16]:
restore_net()

(tensor([[0.1087]], device='cuda:0'), tensor([187.]))
